# 第14章: 継承とポリモーフィズム

## 学習目標
- 継承の概念を理解する
- `extends`キーワードを使ったクラスの継承ができる
- オーバーライドの使い方をマスターする
- ポリモーフィズムの威力を理解する
- 抽象クラスとインターフェースを学ぶ

---

## 🎓 継承の哲学

継承とは、**「既存のクラスの機能を引き継ぎ、拡張する」**仕組みです。

### IS-A関係（「は」の関係）

継承は「IS-A関係」を表します。

- 犬 **は** 動物である
- 教師 **は** 人である  
- 自動車 **は** 乗り物である

```php
// 良い継承の例：IS-A関係
class Dog extends Animal {}  // 犬は動物
class Teacher extends Person {}  // 教師は人
class Car extends Vehicle {}  // 自動車は乗り物

// 悪い継承の例：HAS-A関係（含む関係）
class Engine extends Car {}  // エンジンは車ではない！（間違い）
// 正しくは：車はエンジンを持っている（コンポジション）
```

### 継承のメリット

1. **コードの再利用** - 共通機能を一箇所にまとめる
2. **拡張性** - 新しい機能を追加しやすい
3. **型の安全性** - スーパークラスの型で扱える
4. **多態性（ポリモーフィズム）** - 同じインターフェースで異なる振る舞い

---
## 1. 基本的な継承

`extends`キーワードを使ってクラスを継承します。

In [ ]:
<?php

// 親クラス（スーパークラス・基底クラス）
class Animal {
    protected string $name;
    protected int $age;
    
    public function __construct(string $name, int $age) {
        $this->name = $name;
        $this->age = $age;
        echo "動物 {$name}が生まれました\n";
    }
    
    public function eat(): void {
        echo "{$this->name}は食事をしています\n";
    }
    
    public function sleep(): void {
        echo "{$this->name}は寝ています\n";
    }
    
    public function getInfo(): string {
        return "名前: {$this->name}, 年齢: {$this->age}歳";
    }
    
    public function makeSound(): void {
        echo "{$this->name}は鳴きました\n";
    }
}

// 子クラス（サブクラス・派生クラス）
class Dog extends Animal {
    private string $breed;
    
    public function __construct(string $name, int $age, string $breed) {
        // 親クラスのコンストラクタを呼び出す
        parent::__construct($name, $age);
        $this->breed = $breed;
    }
    
    // メソッドをオーバーライド（上書き）
    public function makeSound(): void {
        echo "{$this->name}はワンワンと吠えました\n";
    }
    
    // 子クラス独自のメソッド
    public function fetch(): void {
        echo "{$this->name}はボールを取りに行きました\n";
    }
    
    public function getInfo(): string {
        return parent::getInfo() . ", 犬種: {$this->breed}";
    }
}

// 別の子クラス
class Cat extends Animal {
    private bool $isIndoor;
    
    public function __construct(string $name, int $age, bool $isIndoor = true) {
        parent::__construct($name, $age);
        $this->isIndoor = $isIndoor;
    }
    
    public function makeSound(): void {
        echo "{$this->name}はニャーと鳴きました\n";
    }
    
    public function scratch(): void {
        echo "{$this->name}は爪を研いでいます\n";
    }
}

// 使用例
$dog = new Dog("ポチ", 3, "柴犬");
$cat = new Cat("タマ", 2, true);

// 親クラスのメソッドも使える
$dog->eat();
$cat->sleep();

// オーバーライドされたメソッド
$dog->makeSound();  // ワンワン
$cat->makeSound();  // ニャー

// 子クラス独自のメソッド
$dog->fetch();
$cat->scratch();

// 親クラスのメソッドを呼び出しつつ拡張
echo $dog->getInfo() . "\n";
echo $cat->getInfo() . "\n";

?>

---
## 2. オーバーライドの詳細

子クラスで親クラスのメソッドを上書きすることを「オーバーライド」と言います。

In [ ]:
<?php

class Employee {
    protected string $name;
    protected int $baseSalary;
    
    public function __construct(string $name, int $baseSalary) {
        $this->name = $name;
        $this->baseSalary = $baseSalary;
    }
    
    public function calculateSalary(): int {
        return $this->baseSalary;
    }
    
    public function work(): void {
        echo "{$this->name}は働いています\n";
    }
    
    public function getDetails(): string {
        return "従業員: {$this->name}";
    }
}

class Manager extends Employee {
    private int $bonus;
    private array $teamMembers = [];
    
    public function __construct(string $name, int $baseSalary, int $bonus) {
        parent::__construct($name, $baseSalary);
        $this->bonus = $bonus;
    }
    
    // 給与計算をオーバーライド
    public function calculateSalary(): int {
        // 親クラスの計算結果にボーナスを追加
        return parent::calculateSalary() + $this->bonus;
    }
    
    // workメソッドを拡張
    public function work(): void {
        // 親クラスのworkを呼び出し、追加の処理を行う
        parent::work();
        echo "チームをマネジメントしています\n";
    }
    
    // getDetailsを完全に置き換え
    public function getDetails(): string {
        $teamCount = count($this->teamMembers);
        return "マネージャー: {$this->name} (チーム: {$teamCount}人)";
    }
    
    public function addTeamMember(Employee $employee): void {
        $this->teamMembers[] = $employee;
    }
    
    public function conductMeeting(): void {
        echo "{$this->name}は会議を主宰しています\n";
    }
}

class Developer extends Employee {
    private array $skills = [];
    
    public function __construct(string $name, int $baseSalary, array $skills) {
        parent::__construct($name, $baseSalary);
        $this->skills = $skills;
    }
    
    public function calculateSalary(): int {
        // スキル数に応じて給与を上乗せ
        $skillBonus = count($this->skills) * 10000;
        return parent::calculateSalary() + $skillBonus;
    }
    
    public function work(): void {
        parent::work();
        echo "コードを書いています\n";
    }
    
    public function getDetails(): string {
        return "開発者: {$this->name} (スキル: " . implode(', ', $this->skills) . ")";
    }
    
    public function codeReview(): void {
        echo "{$this->name}はコードレビューを行っています\n";
    }
}

// 使用例
$manager = new Manager("田中部長", 500000, 100000);
$dev1 = new Developer("鈴木エンジニア", 400000, ["PHP", "JavaScript"]);
$dev2 = new Developer("佐藤エンジニア", 350000, ["Python", "Go", "Docker"]);

// チーム構築
$manager->addTeamMember($dev1);
$manager->addTeamMember($dev2);

// 各従業員の情報と給与
$employees = [$manager, $dev1, $dev2];
foreach ($employees as $emp) {
    echo $emp->getDetails() . "\n";
    echo "  給与: \" . $emp->calculateSalary() . "\n";
    $emp->work();
    echo "\n";
}

?>

---
## 3. ポリモーフィズム（多態性）

ポリモーフィズムとは、**「同じメソッド呼び出しでも、オブジェクトによって異なる振る舞いをすること」**です。

**ポリモーフィズムの威力**
- コードがシンプルになる
- 新しい種類を追加しやすい
- 条件分岐が減る

In [ ]:
<?php

// 支払い方法のインターフェース（後で詳しく説明）
interface PaymentMethod {
    public function pay(int $amount): bool;
    public function getName(): string;
}

class CreditCard implements PaymentMethod {
    private string $cardNumber;
    private string $expiryDate;
    
    public function __construct(string $cardNumber, string $expiryDate) {
        $this->cardNumber = $cardNumber;
        $this->expiryDate = $expiryDate;
    }
    
    public function pay(int $amount): bool {
        echo "クレジットカード({$this->cardNumber})で\{$amount}支払いました\n";
        return true;
    }
    
    public function getName(): string {
        return "クレジットカード";
    }
}

class BankTransfer implements PaymentMethod {
    private string $bankName;
    private string $accountNumber;
    
    public function __construct(string $bankName, string $accountNumber) {
        $this->bankName = $bankName;
        $this->accountNumber = $accountNumber;
    }
    
    public function pay(int $amount): bool {
        echo "銀行振込({$this->bankName})で\{$amount}支払いました\n";
        return true;
    }
    
    public function getName(): string {
        return "銀行振込";
    }
}

class DigitalWallet implements PaymentMethod {
    private string $provider;
    private string $accountId;
    
    public function __construct(string $provider, string $accountId) {
        $this->provider = $provider;
        $this->accountId = $accountId;
    }
    
    public function pay(int $amount): bool {
        echo "電子ウォレット({$this->provider})で\{$amount}支払いました\n";
        return true;
    }
    
    public function getName(): string {
        return "電子ウォレット";
    }
}

// ポリモーフィズムの威力：異なる型を同じように扱う
class PaymentProcessor {
    public function processPayment(PaymentMethod $method, int $amount): bool {
        echo "{$method->getName()}で支払いを処理中...\n";
        return $method->pay($amount);
    }
    
    // 複数の支払い方法をまとめて処理
    public function processMultiple(array $methods, int $amount): void {
        foreach ($methods as $method) {
            $this->processPayment($method, $amount);
        }
    }
}

// 使用例
$creditCard = new CreditCard("****-****-****-1234", "12/28");
$bankTransfer = new BankTransfer("東京銀行", "123-456-789");
$digitalWallet = new DigitalWallet("PayPal", "user@example.com");

$processor = new PaymentProcessor();

// どの支払い方法でも同じように扱える！
$processor->processPayment($creditCard, 5000);
$processor->processPayment($bankTransfer, 10000);
$processor->processPayment($digitalWallet, 3000);

// 配列にまとめて処理
$methods = [$creditCard, $bankTransfer, $digitalWallet];
echo "\n=== 一括支払い処理 ===\n";
$processor->processMultiple($methods, 2500);

?>

---
## 4. 抽象クラス

抽象クラスは、**「直接インスタンス化できないクラス」**です。
共通の機能を定義しつつ、具体的な実装は子クラスに任せたい場合に使います。

**abstractキーワード**
- 抽象クラス：`abstract class`
- 抽象メソッド：`abstract function`（実装を持たないメソッド）

In [ ]:
<?php

// 抽象クラス
abstract class Shape {
    protected string $name;
    protected string $color;
    
    public function __construct(string $name, string $color = "black") {
        $this->name = $name;
        $this->color = $color;
    }
    
    // 具体的なメソッド（実装がある）
    public function getName(): string {
        return $this->name;
    }
    
    public function getColor(): string {
        return $this->color;
    }
    
    public function setColor(string $color): void {
        $this->color = $color;
    }
    
    // 抽象メソッド（実装がない、子クラスで必ず実装する必要がある）
    abstract public function getArea(): float;
    abstract public function getPerimeter(): float;
    
    // 抽象クラスを使ったメソッド（テンプレートメソッドパターン）
    public function describe(): string {
        return "{$this->name} - 色: {$this->color}, " . 
               "面積: {$this->getArea()}, 周囲: {$this->getPerimeter()}";
    }
}

class Circle extends Shape {
    private float $radius;
    
    public function __construct(float $radius, string $color = "red") {
        parent::__construct("円", $color);
        $this->radius = $radius;
    }
    
    // 抽象メソッドの実装（必須）
    public function getArea(): float {
        return M_PI * $this->radius * $this->radius;
    }
    
    public function getPerimeter(): float {
        return 2 * M_PI * $this->radius;
    }
}

class Rectangle extends Shape {
    private float $width;
    private float $height;
    
    public function __construct(float $width, float $height, string $color = "blue") {
        parent::__construct("長方形", $color);
        $this->width = $width;
        $this->height = $height;
    }
    
    public function getArea(): float {
        return $this->width * $this->height;
    }
    
    public function getPerimeter(): float {
        return 2 * ($this->width + $this->height);
    }
}

class Triangle extends Shape {
    private float $base;
    private float $height;
    private float $sideA;
    private float $sideB;
    private float $sideC;
    
    public function __construct(float $base, float $height, float $sideA, float $sideB, float $sideC) {
        parent::__construct("三角形", "green");
        $this->base = $base;
        $this->height = $height;
        $this->sideA = $sideA;
        $this->sideB = $sideB;
        $this->sideC = $sideC;
    }
    
    public function getArea(): float {
        return 0.5 * $this->base * $this->height;
    }
    
    public function getPerimeter(): float {
        return $this->sideA + $this->sideB + $this->sideC;
    }
}

// 使用例
$shapes = [
    new Circle(5),
    new Rectangle(4, 6),
    new Triangle(3, 4, 3, 4, 5)
];

echo "=== 図形一覧 ===\n";
foreach ($shapes as $shape) {
    echo $shape->describe() . "\n";
}

// $shape = new Shape();  // エラー！抽象クラスはインスタンス化できない

?>

---
## 5. インターフェース

インターフェースは、**「クラスが実装すべきメソッドの仕様」**を定義します。

### 抽象クラス vs インターフェース

| | 抽象クラス | インターフェース |
|---|-----------|----------------|
| 実装の保持 | ○ | ×（定数のみ）|
| 継承 | 単一継承のみ | 複数実装可能 |
| 用途 | 共通機能の共有 | 仕様の定義 |
| 状態 | 持てる | 持てない |

In [ ]:
<?php

// インターフェース定義
interface Loggable {
    public function logActivity(): void;
}

interface Cacheable {
    public function getCacheKey(): string;
    public function getCacheDuration(): int;
}

interface Validatable {
    public function validate(): bool;
    public function getErrors(): array;
}

// 複数のインターフェースを実装
class User implements Loggable, Cacheable, Validatable {
    private string $username;
    private string $email;
    private array $errors = [];
    
    public function __construct(string $username, string $email) {
        $this->username = $username;
        $this->email = $email;
    }
    
    // Loggable の実装
    public function logActivity(): void {
        echo "ユーザー {$this->username} のアクティビティをログに記録\n";
    }
    
    // Cacheable の実装
    public function getCacheKey(): string {
        return "user:{$this->username}";
    }
    
    public function getCacheDuration(): int {
        return 3600;  // 1時間
    }
    
    // Validatable の実装
    public function validate(): bool {
        $this->errors = [];
        
        if (strlen($this->username) < 3) {
            $this->errors[] = "ユーザー名は3文字以上である必要があります";
        }
        
        if (!filter_var($this->email, FILTER_VALIDATE_EMAIL)) {
            $this->errors[] = "無効なメールアドレスです";
        }
        
        return empty($this->errors);
    }
    
    public function getErrors(): array {
        return $this->errors;
    }
}

class Product implements Loggable, Cacheable, Validatable {
    private string $name;
    private int $price;
    private array $errors = [];
    
    public function __construct(string $name, int $price) {
        $this->name = $name;
        $this->price = $price;
    }
    
    public function logActivity(): void {
        echo "商品 {$this->name} のアクティビティをログに記録\n";
    }
    
    public function getCacheKey(): string {
        return "product:{$this->name}";
    }
    
    public function getCacheDuration(): int {
        return 7200;  // 2時間
    }
    
    public function validate(): bool {
        $this->errors = [];
        
        if (empty($this->name)) {
            $this->errors[] = "商品名は必須です";
        }
        
        if ($this->price < 0) {
            $this->errors[] = "価格は0以上である必要があります";
        }
        
        return empty($this->errors);
    }
    
    public function getErrors(): array {
        return $this->errors;
    }
}

// インターフェースを使ったポリモーフィズム
function processLoggable(Loggable $item): void {
    $item->logActivity();
}

function cacheItem(Cacheable $item): void {
    echo "キャッシュキー: {$item->getCacheKey()}\n";
    echo "キャッシュ期間: {$item->getCacheDuration()}秒\n";
}

// 使用例
$user = new User("john_doe", "john@example.com");
$product = new Product("ノートPC", 98000);

// 同じインターフェースを実装しているので、同じように扱える
$items = [$user, $product];

echo "=== ログ記録 ===\n";
foreach ($items as $item) {
    processLoggable($item);
}

echo "\n=== キャッシュ処理 ===\n";
foreach ($items as $item) {
    cacheItem($item);
    echo "\n";
}

// バリデーション
echo "=== バリデーション ===\n";
if (!$user->validate()) {
    echo "ユーザーバリデーションエラー:\n";
    foreach ($user->getErrors() as $error) {
        echo "  - {$error}\n";
    }
}

?>

---
## 6. finalキーワード

`final`キーワードを使うと、継承やオーバーライドを禁止できます。

In [ ]:
<?php

// finalクラス：継承禁止
final class Configuration {
    private static array $settings = [];
    
    private function __construct() {
        // インスタンス化禁止
    }
    
    public static function set(string $key, $value): void {
        self::$settings[$key] = $value;
    }
    
    public static function get(string $key, $default = null) {
        return self::$settings[$key] ?? $default;
    }
}

class BaseClass {
    // finalメソッド：オーバーライド禁止
    final public function getId(): string {
        return uniqid('id_', true);
    }
    
    public function process(): void {
        echo "BaseClass::process\n";
    }
}

class DerivedClass extends BaseClass {
    // processはオーバーライドできる
    public function process(): void {
        echo "DerivedClass::process\n";
        parent::process();
    }
    
    // getIdはオーバーライドできない
    // public function getId(): string { ... }  // エラー！
}

// 使用例
$obj = new DerivedClass();
echo "ID: " . $obj->getId() . "\n";  // 親クラスのメソッド
$obj->process();

// class MyConfig extends Configuration {}  // エラー！finalクラスは継承できない

?>

---
## 7. 設計原則：継承よりコンポジション

"**Prefer composition over inheritance**"（継承よりコンポジションを優先する）

### 継承の問題点
- 深い継承階層は理解しにくい
- 多重継承ができない（PHPでは）
""「こんなうちはありません。」
- 親クラスの変更が子クラスに影響する

### コンポジション（合成）のメリット
- 柔軟な組み合わせができる
- 実行時に振る舞いを変えられる
- テストがしやすい

In [ ]:
<?php

/**
 * 機能を小さなクラスに分け、組み合わせる設計
 */

// 機能1：ログ機能
trait Logger {
    private function log(string $message): void {
        echo "[LOG] {$message}\n";
    }
}

// 機能2：キャッシュ機能
trait Cache {
    private array $cache = [];
    
    protected function remember(string $key, callable $callback, int $ttl = 3600) {
        if (isset($this->cache[$key])) {
            echo "キャッシュヒット: {$key}\n";
            return $this->cache[$key];
        }
        echo "キャッシュミス: {$key}\n";
        $result = $callback();
        $this->cache[$key] = $result;
        return $result;
    }
}

// 機能3：バリデーション機能
trait Validator {
    private array $errors = [];
    
    protected function validateEmail(string $email): bool {
        return filter_var($email, FILTER_VALIDATE_EMAIL) !== false;
    }
    
    protected function addError(string $error): void {
        $this->errors[] = $error;
    }
    
    public function getErrors(): array {
        return $this->errors;
    }
}

// クラスに必要な機能だけを組み込む（コンポジション）
class UserRepository {
    use Logger, Cache, Validator;
    
    public function findUser(string $email): ?array {
        return $this->remember("user:{$email}", function() use ($email) {
            $this->log("データベースからユーザーを取得: {$email}");
            // DB取得のシミュレーション
            return ['email' => $email, 'name' => 'Test User'];
        });
    }
    
    public function createUser(string $email, string $name): bool {
        if (!$this->validateEmail($email)) {
            $this->addError("無効なメールアドレス");
            return false;
        }
        
        $this->log("ユーザー作成: {$email}");
        return true;
    }
}

// ログ機能だけ使うクラス
class SimpleLogger {
    use Logger;
    
    public function info(string $message): void {
        $this->log("[INFO] {$message}");
    }
    
    public function error(string $message): void {
        $this->log("[ERROR] {$message}");
    }
}

// 使用例
$repo = new UserRepository();
$user = $repo->findUser("test@example.com");
print_r($user);

// キャッシュから取得
$user = $repo->findUser("test@example.com");

$logger = new SimpleLogger();
$logger->info("アプリケーション開始");
$logger->error("エラーが発生しました");

?>

---
## 📝 実践問題

### 問題1: 図形クラスの拡張

第14章で学んだShapeクラスを拡張してください。

- `Square` クラス（正方形）
- `Ellipse` クラス（楕円）
- `Polygon` 抽象クラス（多角形の基底クラス）
- `RegularPolygon` クラス（正多角形）

### 問題2: 支払いシステムの拡張

新しい支払い方法を追加してください。

- `CryptoPayment` （暗号通貨）
- `PayLater` （後払い）
- `PointPayment` （ポイント支払い）

### 問題3: 通知システム

複数の通知チャネルを持つシステムを作成してください。

- `Notifier` インターフェース
- `EmailNotifier` `SMSNotifier` `PushNotifier` クラス
- `NotificationManager` で複数の通知を一括送信

---
## まとめ

### ✅ ポイントチェック

- [ ] 継承の概念を理解した
- [ ] extendsを使ってクラスを継承できるようになった
- [ ] オーバーライドの使い方をマスターした
- [ ] parent::で親クラスのメソッドを呼べるようになった
- [ ] ポリモーフィズムの威力を理解した
- [ ] 抽象クラスとインターフェースの違いを理解した
- [ ] finalキーワードの使い方を学んだ

### 🎯 継承設計の原則

1. **IS-A関係だけを継承する** - HAS-Aはコンポジションで
2. **深い継承を避ける** - 階層は3層以内に
3. **リスコフの置換原則** - 子クラスは親クラスの代わりになれるべき
4. "**Prefer composition over inheritance**" - 継承よりコンポジションを優先

### 🎉 おめでとうございます！

第11-14章を完了しました！
あなたはオブジェクト指向プログラミングの基礎をマスターしました。

### 🚀 次のステップ

次はさらに高度なトピックに挑戦しましょう！
- 名前空間とオートローディング
- 例外処理
- デザインパターン
- Laravelなどのフレームワーク

---

**関連リソース**
- [PHP: 継承](https://www.php.net/manual/ja/language.oop5.inheritance.php)
- [PHP: 抽象クラス](https://www.php.net/manual/ja/language.oop5.abstract.php)
- [PHP: インターフェース](https://www.php.net/manual/ja/language.oop5.interfaces.php)
- [SOLID原則](https://ja.wikipedia.org/wiki/SOLID)